# Encontrando películas similares

In [13]:
import pandas as pd
r_cols = ['user_id', 'anime_id', 'rating']
df_rating = pd.read_csv('data/rating.csv', usecols=r_cols)

#eliminar los rating -1 que significaria que no han valorado
df_rating = df_rating[df_rating['rating'] != -1]

c_cols = ['anime_id', 'name']
df_anime = pd.read_csv('data/anime.csv', usecols=c_cols)
# Crear un diccionario: {anime_id: name}
anime_dict = dict(zip(df_anime['anime_id'], df_anime['name']))
# Mapear directamente en df_rating
df_rating['name'] = df_rating['anime_id'].map(anime_dict)
df_rating.head()


,user_id,anime_id,rating,name
47,1,8074,10,Highschool of the Dead
81,1,11617,10,High School DxD
83,1,11757,10,Sword Art Online
101,1,15451,10,High School DxD New
153,2,11771,10,Kuroko no Basket


In [15]:
movieRatings = df_rating.pivot(index=['user_id'],columns=['name'], values='rating')
movieRatings.head()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
xxxHolic = movieRatings['Gintama']
xxxHolic.head()

user_id
1    NaN
2    NaN
3    NaN
4    NaN
5    9.0
Name: Gintama, dtype: float64

"corrwith" hace el cálculo de correlación entre Star Wars con todas las otras películas.

In [ ]:
similarxxxHolic = movieRatings.corrwith(xxxHolic)
similarxxxHolic = similarxxxHolic.dropna()

df = pd.DataFrame(similarxxxHolic)
df.head(10)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3057: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:2914: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:2914: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/numpy/lib/_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,0
name,
0,0.892029
001,1.000000
009 Re:Cyborg,0.671861
009-1,0.702477
009-1: R&amp;B,0.601229
00:08,0.289295
07-Ghost,0.618813
1+2=Paradise,0.760680
1000-nen Joou: Queen Millennia,0.936617


In [ ]:
similarxxxHolic.sort_values(ascending=False).head(40)

name
001                                                                1.0
Zukkoke Knight: Don De La Mancha                                   1.0
Doraemon Movie 35: Nobita no Space Heroes                          1.0
Doraemon: Nobita to Mirai Note                                     1.0
Detective Conan: Episode One - Chiisaku Natta Meitantei            1.0
Detective Conan: Nazo no Suisei Kaijuu wo Oe!                      1.0
Yukiwatari                                                         1.0
Yume kara, Samenai                                                 1.0
Yume no Crayon Oukoku                                              1.0
Yuusei Shounen Papii                                               1.0
Yuusha Shirei Dagwon: Suishou no Hitomi no Shounen                 1.0
77Danui Bimil                                                      1.0
Doudou                                                             1.0
Doukyuusei 2 (OVA)                                                 1.0
D

Hay muchas películas que en realidad tienen poco que ver con Star Wards. Probablemente hay un par de personas que vieron Star Wars y también Full Speed por ejemplo, que no es una película muy conocida y eso ya le da un puntaje de similaridad alto. ¿Cómo se puede arreglar?

Agrupamos por título y mostramos la cantidad de calificaciones y su media.

In [ ]:
import numpy as np

movieStatsNuevo = df_rating.groupby('name').agg({'rating': np.size})
movieStatsNuevo.head(15)

,rating
name,
0,26
0-sen Hayato,1
0-sen Hayato Pilot,1
001,12
009 Re:Cyborg,291
009-1,304
009-1: R&amp;B,128
00:08,43
07-Ghost,4867


Eliminamos las películas que tienen menos de 100 calificaciones porque no queremos que los datos estén sesgados para algunas personas que vieron películas que nadie conoce y después se recomienden.

In [ ]:
popularMoviesNuevo = movieStatsNuevo['rating'] >= 100
movieStatsNuevo[popularMoviesNuevo].sort_values([('rating')], ascending=False)[:20]

,rating
name,
Death Note,39340
Sword Art Online,30583
Shingeki no Kyojin,29584
Code Geass: Hangyaku no Lelouch,27718
Elfen Lied,27506
Angel Beats!,27183
Naruto,25925
Fullmetal Alchemist,25032
Fullmetal Alchemist: Brotherhood,24574


El límite de 100 calificaciones quizás es muy bajo, pero ya nos sirve. Ahora unimos estos datos con el dataframe de película similares a StarWars.

In [ ]:
df = movieStatsNuevo[popularMoviesNuevo].join(pd.DataFrame(similarxxxHolic, columns=['similarity']))
df.head()


,rating,similarity
name,,
009 Re:Cyborg,291,0.671861
009-1,304,0.702477
009-1: R&amp;B,128,0.601229
07-Ghost,4867,0.618813
1+2=Paradise,143,0.760680


Ahora mostrar ordenados por similaridad.

In [ ]:
df.sort_values(['similarity'], ascending=False).head(20)
#pd.set_option('display.max_rows', None)
#print(df)

,rating,similarity
name,,
Gintama,8647,1.000000
Elf no Wakaokusama,109,0.971710
Ojamajo Doremi Na-i-sho,135,0.968547
Waga Seishun no Arcadia,164,0.967030
Samurai Spirits: Haten Gouma no Shou,101,0.965362
Harlock Saga: Nibelung no Yubiwa,111,0.957944
Shin Getter Robo,155,0.957014
Meitantei Holmes,121,0.956978
Maria†Holic Alive Special,120,0.949999
